In [4]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
from sklearn.utils import shuffle
import tensorflow as tf

In [5]:
TRAIN_DIR = r"C:\Users\Nitin Singh\Desktop\Dogs-Vs-Cats-Kaggle-master\data\train"
TEST_DIR = r"C:\Users\Nitin Singh\Desktop\Dogs-Vs-Cats-Kaggle-master\data\test"
IMG_SIZE = 50
LR0 = 0.001

In [6]:
def label_img(img):
    word_label = img.split('.')[-3]

    if word_label == 'cat': return [1,0]

    elif word_label == 'dog': return [0,1]
    
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [7]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

In [8]:
train_data = create_train_data()
test_data = process_test_data()

100%|███████████████████████████████████████████████████████████████████████████| 12500/12500 [00:31<00:00, 401.90it/s]


In [9]:
LR = 0.001
MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic') # just so we remember which saved model is which, sizes must match
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
IMG_SIZE = 50
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [10]:
import tensorflow as tf
tf.reset_default_graph()

In [11]:
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = dropout(convnet, 0.9)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = dropout(convnet, 0.9)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = dropout(convnet, 0.8)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

    
train = train_data[:-500]
test = train_data[-500:]


X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]
model.fit({'input': X}, {'targets': Y}, n_epoch=50, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 19149  | total loss: 0.17756 | time: 44.258s
| Adam | epoch: 050 | loss: 0.17756 - acc: 0.9276 -- iter: 24448/24500
Training Step: 19150  | total loss: 0.18881 | time: 45.373s
| Adam | epoch: 050 | loss: 0.18881 - acc: 0.9223 | val_loss: 0.47131 - val_acc: 0.8400 -- iter: 24500/24500
--
